# Create Heat Map for significant RNA Transport genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,84/134,3.960162e-41,1.160328e-38,0,0,4.801729,446.703392,RBM25;EIF4A3;DDX42;HNRNPU;PRPF19;USP39;EFTUD2;...,KEGG_2016
1,RNA transport Homo sapiens hsa03013,76/172,6.887335e-24,1.008995e-21,0,0,3.384607,180.509081,CYFIP2;NUP107;NUP188;RPP30;EIF4A3;PHAX;PNN;XPO...,KEGG_2016
2,DNA replication Homo sapiens hsa03030,22/36,1.947423e-11,1.901983e-09,0,0,4.681050,115.443730,RFC5;FEN1;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;PRIM1;...,KEGG_2016
3,Mismatch repair Homo sapiens hsa03430,16/23,6.740173e-10,4.937177e-08,0,0,5.328626,112.528681,RFC5;RFC3;PCNA;RFC4;RFC1;RFC2;RPA1;MLH1;POLD3;...,KEGG_2016
4,mRNA surveillance pathway Homo sapiens hsa03015,34/91,3.983668e-09,2.334429e-07,0,0,2.861941,55.352977,SMG1;RNMT;EIF4A3;MSI1;PPP2R2A;MSI2;SMG7;SMG6;P...,KEGG_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 1
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

RNA transport Homo sapiens hsa03013
total genes: 76


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [8]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

76

In [9]:
# Only include p-values < a certain cutoff
a = 0.01
plot_df = plot_df.loc[plot_df['P_Value'] <= a]

In [10]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 1500)

Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [11]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [12]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['NUP37', 'NUP153', 'XPO5', 'FXR1', 'NUP210', 'SENP2'], dtype=object)

In [15]:
# Only include p-values < a 
genes_pn = genes_pn.loc[genes_pn['P_Value'] <= a]

In [17]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=400)

Loading BokehJS ...

# Sig in multiple cancers

In [18]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [23]:
enr2.res2d.iloc[[4]]

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
4,RNA transport Homo sapiens hsa03013,15/172,8.324097e-08,0.000005,0,0,5.608315,91.424098,RANBP2;NUP210;NUP155;NCBP1;NUP133;THOC3;THOC2;...,KEGG_2016


In [24]:
index2 = 4
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

RNA transport Homo sapiens hsa03013
total genes: 15


In [25]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['NCBP1', 'TPR', 'THOC2', 'XPO1', 'XPO5', 'RANBP2', 'NUP155',
       'EIF5', 'THOC3', 'THOC7', 'NUP85', 'FXR1', 'RAE1', 'NUP133',
       'NUP210'], dtype=object)

In [26]:
# Only include p-values < a 
plot_df2 = plot_df2.loc[plot_df2['P_Value'] <= a]

In [28]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=600)

Loading BokehJS ...

In [29]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['XPO5', 'FXR1', 'NUP210'], dtype=object)

In [30]:
# Only include p-values < a 
genes_pn_mult = genes_pn_mult.loc[genes_pn_mult['P_Value'] <= a]

In [33]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=300, plot_width=250)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [34]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['XPO5', 'FXR1', 'NUP210']


In [35]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,RNA transport Homo sapiens hsa03013,3/172,6.250379e-07,0.000183,0,0,116.27907,1661.099243,FXR1;NUP210;XPO5,KEGG_2016


In [23]:
# Look at p-values 
certain_gene = 'XPO5'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
803,XPO5,0.000910,0.473772,Gbm
11306,XPO5,0.000277,0.511937,Hnscc
30268,XPO5,0.624771,-0.064600,Luad
33647,XPO5,0.002771,0.707800,Lscc
44795,XPO5,0.033289,0.623800,Brca
59790,XPO5,0.586051,-0.025352,Ov
63943,XPO5,0.000011,-0.505300,En
82205,XPO5,0.930620,0.074500,Colon
